In [7]:
# import des librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# clean text data
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

# # prep the data for the model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate


# # model
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate

In [8]:
# import des données
jobs_df = pd.read_csv("../data/gold_salaires_data.csv")
jobs_salary = pd.read_csv("../data/job_salary.csv")
company_salary = pd.read_csv("../data/company_salary.csv")
jobs_df.head()

,code,ville,day,month,year,date,jobs,description,company_description,experience,qualifications,industry,company
0,75,Paris (Dept.),16,7,2023,16/07/2023,responsable data h/f,"Votre mission Au sein du service data, vous ê...","Colas, filiale du groupe Bouygues, a pour miss...",Expérience exigée (de 3 à 5 ans),NaN,NaN,NaN
1,54,Meurthe et Moselle,16,7,2023,16/07/2023,data analyst,"Jean, consultant spécialisé sur les métiers de...","Basé au cœur du Grand-Est, le cabinet Harry Ho...",Débutant accepté,NaN,NaN,NaN
2,35,Ille et Vilaine,16,7,2023,16/07/2023,data manager,"VINCI Facilities (11 M€ de CA, 50 collaborateu...","Dans un monde en évolution permanente, VINCI E...",Expérience exigée,NaN,NaN,NaN
3,35,Ille et Vilaine,16,7,2023,16/07/2023,chef de projet data f/h (h/f),"VINCI Facilities (11 M€ de CA, 50 collaborateu...","Dans un monde en évolution permanente, VINCI E...",Expérience exigée,NaN,NaN,NaN
4,92,Hauts de Seine,16,7,2023,16/07/2023,documentaliste et controleur data h/f,Osez exprimer qui vous êtes vraiment !\n\nDire...,Découvrez votre nouveau projet professionnel !...,Expérience exigée,NaN,NaN,NaN


In [9]:
jobs_salary.head()

,Job,salary($),salary_type,Base Salary,Bonus,Total Pay,Profit Sharing,Commission,Hourly Rate
0,ABAP Developer,90000.0,year,$68k - $120k,$2k - $20k,$67k - $132k,NaN,NaN,NaN
1,ASIC Design Engineer,108330.0,year,$83k - $148k,$1k - $15k,$85k - $164k,$2k - $15k,NaN,NaN
2,ASIC Engineer,116546.0,year,$59k - $165k,$503 - $22k,$59k - $179k,NaN,NaN,NaN
3,ASP.NET Developer,71152.0,year,$50k - $94k,$993 - $10k,$47k - $96k,$2k - $5k,NaN,NaN
4,Actuarial Associate,82113.0,year,$65k - $101k,$2k - $13k,$67k - $109k,$2k - $7k,NaN,NaN


In [10]:
company_salary.head()

,company,avg_salary($),salary_type
0,A.B._Data%2C_Ltd.,NaN,year
1,A-Tech,20.0,hour
2,A_Wireless,NaN,year
3,A_%26_A_Telecom,83.0,year
4,A_%26_T_Systems,NaN,year


## Nettoyage et préparation des données

In [11]:
# préparer un df avec uniquement les colonnes à utiliser pour le modèle
df = jobs_df[["jobs", "description", "experience", "company", "company_description"]]
df.head()

,jobs,description,experience,company,company_description
0,responsable data h/f,"Votre mission Au sein du service data, vous ê...",Expérience exigée (de 3 à 5 ans),NaN,"Colas, filiale du groupe Bouygues, a pour miss..."
1,data analyst,"Jean, consultant spécialisé sur les métiers de...",Débutant accepté,NaN,"Basé au cœur du Grand-Est, le cabinet Harry Ho..."
2,data manager,"VINCI Facilities (11 M€ de CA, 50 collaborateu...",Expérience exigée,NaN,"Dans un monde en évolution permanente, VINCI E..."
3,chef de projet data f/h (h/f),"VINCI Facilities (11 M€ de CA, 50 collaborateu...",Expérience exigée,NaN,"Dans un monde en évolution permanente, VINCI E..."
4,documentaliste et controleur data h/f,Osez exprimer qui vous êtes vraiment !\n\nDire...,Expérience exigée,NaN,Découvrez votre nouveau projet professionnel !...


In [12]:
df["jobs"] = df["jobs"].str.replace(" h/f", "").replace(" (h/f)", "")
df["jobs"] = df["jobs"].str.replace(" - f/h/x", "")
df.head()

C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_400\1368623702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["jobs"] = df["jobs"].str.replace(" h/f", "").replace(" (h/f)", "")
C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_400\1368623702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["jobs"] = df["jobs"].str.replace(" - f/h/x", "")


,jobs,description,experience,company,company_description
0,responsable data,"Votre mission Au sein du service data, vous ê...",Expérience exigée (de 3 à 5 ans),NaN,"Colas, filiale du groupe Bouygues, a pour miss..."
1,data analyst,"Jean, consultant spécialisé sur les métiers de...",Débutant accepté,NaN,"Basé au cœur du Grand-Est, le cabinet Harry Ho..."
2,data manager,"VINCI Facilities (11 M€ de CA, 50 collaborateu...",Expérience exigée,NaN,"Dans un monde en évolution permanente, VINCI E..."
3,chef de projet data f/h (h/f),"VINCI Facilities (11 M€ de CA, 50 collaborateu...",Expérience exigée,NaN,"Dans un monde en évolution permanente, VINCI E..."
4,documentaliste et controleur data,Osez exprimer qui vous êtes vraiment !\n\nDire...,Expérience exigée,NaN,Découvrez votre nouveau projet professionnel !...


In [22]:
df.dtypes

jobs                   object
description            object
experience             object
company                object
company_description    object
dtype: object

In [16]:
# Prétraitement des données
def preprocess_text(text):
    # Suppression des caractères spéciaux et de la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Conversion en minuscules
    text = text.lower()
    
    # Suppression des mots vides
    stop_words = set(stopwords.words('french'))  
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoindre les tokens prétraités en une seule chaîne
    processed_text = ' '.join(tokens)
    
    return processed_text

In [17]:
df["description"][0]

"Votre mission  Au sein du service data, vous êtes en charge de la définition, du déploiement et de la coordination des projets data / IT de la Business Unit en lien avec ses priorités stratégiques. Dans ce cadre, vos principales missions sont les suivantes : - Définir et mettre en œuvre la stratégie informatique et Data de la BU qui doit venir en soutien des priorités stratégiques de la business unit. - Vous assurez la Data Visualisation : Vous vous occupez de la restitution graphique des résultats avec les logiciels Power BI - Restitution : présentation des résultats et recommandations sur leurs utilisations - Vous travaillez avec les équipes Achats sur l'intégration des données Carbone fournisseurs dans les outils digitaux - Vous assurez le Data Management : vous recherchez des données les plus pertinentes, vous vous occupez de la création de flux et de bases de données d'étude - Vous assurez l'interface avec les équipes financières et les équipes digitales - Vous vous occupez du dé

In [32]:
df["description"] = df["description"].apply(preprocess_text)
df["company_description"] = df["company_description"].astype(str)
df["company_description"] = df["company_description"].apply(preprocess_text)
df["experience"] = df["experience"].apply(preprocess_text)

C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_400\3973741798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["description"] = df["description"].apply(preprocess_text)
C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_400\3973741798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["company_description"] = df["company_description"].astype(str)
C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_400\3973741798.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [31]:
df["experience"][1]

'débutant accepté'

In [33]:
df.head()

,jobs,description,experience,company,company_description
0,responsable data,mission sein service data charge définition dé...,expérience exigée 3 5 an,NaN,cola filiale groupe bouygues a mission dimagin...
1,data analyst,jean consultant spécialisé métiers lit lorrain...,débutant accepté,NaN,basé cœur grandest cabinet harry hope déploie ...
2,data manager,vinci facility 11 ca 50 collaborateurs recherc...,expérience exigée,NaN,monde évolution permanente vinci energy accélè...
3,chef de projet data f/h (h/f),vinci facility 11 ca 50 collaborateurs recherc...,expérience exigée,NaN,monde évolution permanente vinci energy accélè...
4,documentaliste et controleur data,osez exprimer vraiment directement rattachée m...,expérience exigée,NaN,découvrez nouveau projet professionnel comme e...


## Model

In [ ]:
# Combine all your text data for effective tokenization
all_text = skills + location + contract_type + job_description + job_title

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)

# Convert text data to sequences
skills_seq = tokenizer.texts_to_sequences(skills)
location_seq = tokenizer.texts_to_sequences(location)
contract_seq = tokenizer.texts_to_sequences(contract_type)
description_seq = tokenizer.texts_to_sequences(job_description)
title_seq = tokenizer.texts_to_sequences(job_title)

In [ ]:
MAXLEN = 300  # You can adjust this based on your data

skills_seq_padded = pad_sequences(skills_seq, maxlen=MAXLEN)
location_seq_padded = pad_sequences(location_seq, maxlen=MAXLEN)
contract_seq_padded = pad_sequences(contract_seq, maxlen=MAXLEN)
description_seq_padded = pad_sequences(description_seq, maxlen=MAXLEN)
title_seq_padded = pad_sequences(title_seq, maxlen=MAXLEN)

In [ ]:
EMBEDDING_DIM = 100  # or the size of your embeddings if you're using pre-trained
VOCAB_SIZE = len(tokenizer.word_index) + 1  # +1 for padding

# Define an input sequence and process it
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(encoder_inputs)

# You might want separate encoders for each input feature or one concatenated feature
# For simplicity, I'll use one encoder for demonstration
encoder_lstm = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Set up the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & train
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([skills_seq_padded, title_seq_padded], description_seq_padded, batch_size=64, epochs=10)